In [1]:
import json
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Get GDP data
- For all "Low & middle income" (LMY) countries
- Source for data file: https://data.worldbank.org/indicator/NY.GDP.MKTP.CD?locations=XO
- Here we format the data to fit the one produced by  `debt_forecasts.ipynb`
- We are only interested in the latest data. Since we are comparing against a predicted number.

In [3]:
gdp_df = pd.read_csv("gdp_lmy.csv")

# drop unwanted cols
gdp_df = gdp_df.drop(['Country Code', 'Indicator Name','Indicator Code','Unnamed: 66'], axis=1)

gdp_df = gdp_df.set_index("Country Name")
gdp_df.index.name = None
gdp_df = gdp_df.transpose()

# convert index to datetime
#gdp_df.index.name = "date"
gdp_df.index = pd.to_datetime(gdp_df.index,format='%Y')

# resample to fit the debt data
gdp_df = gdp_df.resample("AS").mean()

# fill NaNs
gdp_df = gdp_df.fillna(method="bfill")
gdp_df = gdp_df.fillna(method="ffill")

# drop cols and rows that only contain NaNs
gdp_df = gdp_df.dropna(how='all', axis=1)
gdp_df = gdp_df.dropna(how='all', axis=0)

# convert column types from np.float64 to np.int64
gdp_df = gdp_df.astype(np.int64)

# enforce sort by date
gdp_df = gdp_df.sort_index()
gdp_df

,Aruba,Africa Eastern and Southern,Afghanistan,Africa Western and Central,Angola,Albania,Andorra,Arab World,United Arab Emirates,Argentina,Armenia,American Samoa,Antigua and Barbuda,Australia,Austria,Azerbaijan,Burundi,Belgium,Benin,Burkina Faso,Bangladesh,Bulgaria,Bahrain,"Bahamas, The",Bosnia and Herzegovina,Belarus,Belize,Bermuda,Bolivia,Brazil,Barbados,Brunei Darussalam,Bhutan,Botswana,Central African Republic,Canada,Central Europe and the Baltics,Switzerland,Channel Islands,Chile,China,Cote d'Ivoire,Cameroon,"Congo, Dem. Rep.","Congo, Rep.",Colombia,Comoros,Cabo Verde,Costa Rica,Caribbean small states,Cuba,Curacao,Cayman Islands,Cyprus,Czech Republic,Germany,Djibouti,Dominica,Denmark,Dominican Republic,Algeria,East Asia & Pacific (excluding high income),Early-demographic dividend,East Asia & Pacific,Europe & Central Asia (excluding high income),Europe & Central Asia,Ecuador,"Egypt, Arab Rep.",Euro area,Eritrea,Spain,Estonia,Ethiopia,European Union,Fragile and conflict affected situations,Finland,Fiji,France,Faroe Islands,"Micronesia, Fed. Sts.",Gabon,United Kingdom,Georgia,Ghana,Guinea,"Gambia, The",Guinea-Bissau,Equatorial Guinea,Greece,Grenada,Greenland,Guatemala,Guam,Guyana,High income,"Hong Kong SAR, China",Honduras,Heavily indebted poor countries (HIPC),Croatia,Haiti,Hungary,IBRD only,IDA & IBRD total,IDA total,IDA blend,Indonesia,IDA only,Isle of Man,India,Ireland,"Iran, Islamic Rep.",Iraq,Iceland,Israel,Italy,Jamaica,Jordan,Japan,Kazakhstan,Kenya,Kyrgyz Republic,Cambodia,Kiribati,St. Kitts and Nevis,"Korea, Rep.",Kuwait,Latin America & Caribbean (excluding high income),Lao PDR,Lebanon,Liberia,Libya,St. Lucia,Latin America & Caribbean,Least developed countries: UN classification,Low income,Liechtenstein,Sri Lanka,Lower middle income,Low & middle income,Lesotho,Late-demographic dividend,Lithuania,Luxembourg,Latvia,"Macao SAR, China",Morocco,Monaco,Moldova,Madagascar,Maldives,Middle East & North Africa,Mexico,Marshall Islands,Middle income,North Macedonia,Mali,Malta,Myanmar,Middle East & North Africa (excluding high income),Montenegro,Mongolia,Northern Mariana Islands,Mozambique,Mauritania,Mauritius,Malawi,Malaysia,North America,Namibia,New Caledonia,Niger,Nigeria,Nicaragua,Netherlands,Norway,Nepal,Nauru,New Zealand,OECD members,Oman,Other small states,Pakistan,Panama,Peru,Philippines,Palau,Papua New Guinea,Poland,Pre-demographic dividend,Puerto Rico,Portugal,Paraguay,West Bank and Gaza,Pacific island small states,Post-demographic dividend,French Polynesia,Qatar,Romania,Russian Federation,Rwanda,South Asia,Saudi Arabia,Sudan,Senegal,Singapore,Solomon Islands,Sierra Leone,El Salvador,San Marino,Somalia,Serbia,Sub-Saharan Africa (excluding high income),South Sudan,Sub-Saharan Africa,Small states,Sao Tome and Principe,Suriname,Slovak Republic,Slovenia,Sweden,Eswatini,Sint Maarten (Dutch part),Seychelles,Syrian Arab Republic,Turks and Caicos Islands,Chad,East Asia & Pacific (IDA & IBRD countries),Europe & Central Asia (IDA & IBRD countries),Togo,Thailand,Tajikistan,Turkmenistan,Latin America & the Caribbean (IDA & IBRD countries),Timor-Leste,Middle East & North Africa (IDA & IBRD countries),Tonga,South Asia (IDA & IBRD),Sub-Saharan Africa (IDA & IBRD countries),Trinidad and Tobago,Tunisia,Turkey,Tuvalu,Tanzania,Uganda,Ukraine,Upper middle income,Uruguay,United States,Uzbekistan,St. Vincent and the Grenadines,"Venezuela, RB",Virgin Islands (U.S.),Vietnam,Vanuatu,World,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
1960-01-01,405463417,20082715853,537777811,10404280784,5930503400,1857338011,78619206,35087069740,14720672506,24450604877,2256838857,512000000,77496753,18606786874,6592693841,446305555,195999990,11658722590,226195579,330442817,4274893913,19839230769,3072698328,169803921,1255802469,21650000000,28071888,84466654,373879363,17030465539,311809337,114040245,128717410,30412308,112155598,40461721692,256335875977,9522746719,5945677376,4110000000,59716467625,546203561,614206071,3359404117,131731862,4031152976,2122

# Get debt predictions
- Goal is to find a subset of `gdp_df` that contains data of countries that belong to `gdp_df` and `preds_df`.


In [4]:
# get country names in wb_df
preds_df = pd.read_csv("debt_predictions.csv",index_col="date", infer_datetime_format=True, parse_dates=True)
preds_df

,Afghanistan,Angola,Albania,Argentina,Armenia,Azerbaijan,Burundi,Benin,Burkina Faso,Bangladesh,Bulgaria,Bosnia and Herzegovina,Belarus,Belize,Bolivia,Brazil,Bhutan,Botswana,Central African Republic,China,Cote d'Ivoire,Cameroon,"Congo, Dem. Rep.","Congo, Rep.",Colombia,Comoros,Cabo Verde,Costa Rica,Djibouti,Dominica,Dominican Republic,Algeria,Ecuador,"Egypt, Arab Rep.",Eritrea,Ethiopia,Fiji,Gabon,Georgia,Ghana,Guinea,"Gambia, The",Guinea-Bissau,Grenada,Guatemala,Guyana,Honduras,Haiti,Indonesia,India,"Iran, Islamic Rep.",Jamaica,Jordan,Kazakhstan,Kenya,Kyrgyz Republic,Cambodia,Lao PDR,Lebanon,Liberia,St. Lucia,Sri Lanka,Lesotho,Morocco,Moldova,Madagascar,Maldives,Mexico,North Macedonia,Mali,Myanmar,Montenegro,Mongolia,Mozambique,Mauritania,Mauritius,Malawi,Niger,Nigeria,Nicaragua,Nepal,Pakistan,Panama,Peru,Philippines,Papua New Guinea,Paraguay,Romania,Russian Federation,Rwanda,Sudan,Senegal,Solomon Islands,Sierra Leone,El Salvador,Somalia,Serbia,Sao Tome and Principe,Eswatini,Syrian Arab Republic,Chad,Togo,Thailand,Tajikistan,Turkmenistan,Timor-Leste,Tonga,Tunisia,Turkey,Tanzania,Uganda,Ukraine,Uzbekistan,St. Vincent and the Grenadines,Vietnam,Vanuatu,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,3036369312,67286794415,10859239096,253760056780,13093864395,15811923784,625985283,5250479649,4494781776,67749043920,43347397245,14263489883,42491219492,1526004210,15421072053,549234311520,2868979841,1597427934,935630034,2349388621678,25072693104,13863899165,6137246433,5253134974,155171767040,301131309,2069786254,31287627814,2678724034,328975990,44467742286,5178215526,56190547675,131579729752,788537429,30364411863,1518014488,7615875628,20088956454,31323084616,4175376873,775654200,805942506,658742963,25136104794,1506687262,11016615220,2317684493,417531650069,564179014212,5451188395,18034588389,38016104454,162974379888,38193646113,8697096132,17562061636,17164137036,68865417339,1480598762,733837563,56341952149,1052218388,65682954942,8475902591,4873178319,3351712547,467511920765,10606666542,6081077523,13348140041,9722712647,33236211399,20932324154,5710104834,18527475048,2943281966,4590696470,70570530052,12050448081,7904918393,116505539223,108866240274,73549205266,98479781366,17970999839,19763367609,142389651029,475518171675,8193366972,22953534423,17238618348,428927088,2113502029,18333708634,4659652464,38467088092,291188749,766084561,4763248268,3654297383,2546422966,204146618941,6797818017,5636041535,231800534,194304203,41038099975,435889447921,25537812220,17206774508,129899079724,32174405287,406295764,125045334156,455667428,437296399,3108497520,7120836024,170766641061,30045885685,12740664732
2021-01-01,3172930266,69530776214,11476099065,241444180159,15284206734,15803531552,629565718,5957167841,4774252168,78004340328,45211546620,14738719070,43504080869,1549715569,15967945909,537613406194,3103212020,1603324352,952526223,2160293737009,28899189480,14446429014,6159980847,5249887121,165163863867,303942481,2306914507,33196590836,2864549822,353190079,49044613038,5034495139,60318799545,147768832454,789727358,31529949195,1658266377,7905471975,20992880752,35400017612,4221738070,772537963,924895337,647357451,25198130557,1505998126,11811581361,2359750114,429732809167,593924926978,5615117156,18419779722,40475939875,164941609494,42938769335,8832224495,19786107278,17537980446,66916982783,1602665337,703197563,56506189854,1081454623,70617670983,8597105057,4987376507,3812596958,464658228320,11160616654,6589083181,14452083655,10576255946,34630367048,21577958950,5759823451,15650270032,3056980405,4964139281,79177306829,12214513591,9210940200,123518263695,115408200710,74860076380,101125063323,17628991472,24771324809,158388028222,473457544097,9494905696,23238374322,19194125375,460163234,2206702867,18877131961,4377093798,41402735079,295483693,719869444,4763740756,3625026666,2956367339,219463764566,7135531205,5102048128,2

In [5]:
# find intersecting countries
# list countries
preds_df_countries = preds_df.columns.unique()
# get new dataframe by filtering columns by name and handling non-existent
intersection_df = gdp_df.loc[:, gdp_df.columns.isin(preds_df_countries)]

In [6]:
# test that column names match
intersection_df.columns == preds_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [8]:
# get latest observations
gdp_data = pd.DataFrame(intersection_df.loc[gdp_df.index.max()])
gdp_data = gdp_data.transpose()
gdp_data

,Afghanistan,Angola,Albania,Argentina,Armenia,Azerbaijan,Burundi,Benin,Burkina Faso,Bangladesh,Bulgaria,Bosnia and Herzegovina,Belarus,Belize,Bolivia,Brazil,Bhutan,Botswana,Central African Republic,China,Cote d'Ivoire,Cameroon,"Congo, Dem. Rep.","Congo, Rep.",Colombia,Comoros,Cabo Verde,Costa Rica,Djibouti,Dominica,Dominican Republic,Algeria,Ecuador,"Egypt, Arab Rep.",Eritrea,Ethiopia,Fiji,Gabon,Georgia,Ghana,Guinea,"Gambia, The",Guinea-Bissau,Grenada,Guatemala,Guyana,Honduras,Haiti,Indonesia,India,"Iran, Islamic Rep.",Jamaica,Jordan,Kazakhstan,Kenya,Kyrgyz Republic,Cambodia,Lao PDR,Lebanon,Liberia,St. Lucia,Sri Lanka,Lesotho,Morocco,Moldova,Madagascar,Maldives,Mexico,North Macedonia,Mali,Myanmar,Montenegro,Mongolia,Mozambique,Mauritania,Mauritius,Malawi,Niger,Nigeria,Nicaragua,Nepal,Pakistan,Panama,Peru,Philippines,Papua New Guinea,Paraguay,Romania,Russian Federation,Rwanda,Sudan,Senegal,Solomon Islands,Sierra Leone,El Salvador,Somalia,Serbia,Sao Tome and Principe,Eswatini,Syrian Arab Republic,Chad,Togo,Thailand,Tajikistan,Turkmenistan,Timor-Leste,Tonga,Tunisia,Turkey,Tanzania,Uganda,Ukraine,Uzbekistan,St. Vincent and the Grenadines,Vietnam,Vanuatu,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
2021-01-01,20116137325,58375976292,14887629268,389288056265,12641209802,42607176470,2841786382,15651545331,17933606353,323056957972,69889347433,19946496562,60258239055,1636280797,36572764862,1444733258971,2315437338,15061922801,2380087758,14722730697890,61348579465,40804449726,48716960860,10187122341,271437596293,1235400352,1703698676,61846895120,3384385216,504214814,78844702329,145009181490,98808010000,365252651278,2065001626,107645054311,4533883782,15316826191,15846489610,68532281805,15681050917,1868086274,1431758242,1042100555,77604632170,5471256594,23662231633,14508218017,1058423838345,2660245248867,203471303952,13812425036,43697659295,171082379532,101013726529,7735976272,25808561550,19132635711,31735217784,3201187800,1616772740,80676681933,1875227642,114725065285,11915547262,13056079982,3742769967,1073915880822,12263710122,17465392915,79852046610,4769860740,13312981594,14019446610,7913680231,10920606197,12182348212,13741378450,432293776262,12621505382,33657175561,262610002938,53977036995,202014363787,361489325230,24668899683,35670301496,248715551366,1483497784867,10333991455,21329109521,24644234594,1545888426,4063289449,24638720000,6965285324,53335016425,472914469,3972728948,22777882165,10829076801,7574636978,501643653514,8194150301,45231428571,1902156800,488829964,41620349986,719954821683,62409709110,37600368180,155498989149,59929951114,807474074,271158442448,881547928,807100820,7716925356,21606140907,335442101366,18110631358,18051170798


# Export data

In [10]:
# save json
gdp_json = gdp_data.to_json(orient= "index")
with open('gdp.json', 'w') as outfile:
    json.dump(gdp_json, outfile)